# [Module 5] Create Dataset Group


* IAM Role 생성 및 권한 할당
    - Personalize 서비스가 사용할 역할을 생성 및 권한을 할당 합니다.
* 데이타 세트 그룹 생성 (DatasetGroup)    
* 데이타 스키마 생성
* 데이타 세트 생성 (Dataset)
* 데이타 Import (S3 --> Personalize 서비스로 다운로드)

---

# 0. 환경 설정

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdate
from botocore.exceptions import ClientError

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
suffix = str(np.random.uniform())[4:9]

In [19]:
%store -r

## 1. Personalize Service의 S3 접근 권한
Personalize Service 는 해당 S3 버킷에 접근하기 위해서 권한

In [5]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*",
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'MKGYA1AKR00JSK7S',
  'HostId': 'drxP32ueZyOi5i2GsV/5/1dGZZHW/75HfLGi5EvpxnhiZxgDPtIgHYXUYIoWy7ggMZ0L6Yphbso=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'drxP32ueZyOi5i2GsV/5/1dGZZHW/75HfLGi5EvpxnhiZxgDPtIgHYXUYIoWy7ggMZ0L6Yphbso=',
   'x-amz-request-id': 'MKGYA1AKR00JSK7S',
   'date': 'Tue, 28 Feb 2023 11:42:09 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Personalize IAM Role

또한, Amazon Personalize는 특정 작업들을 실행할 권한을 갖기 위해, AWS에서 역할을 맡을 수 있는 기능이 필요합니다. 
예를 들어 Personalize 는 S3에 접근을 해야 합니다. 그래서 이를 위한 역할이 필요하고, 이 역할은 S3 접근 권한이 필요 합니다.

In [6]:
# 이미 기존에 사용하신 Role 이 있으면 아래와 Role ARN 을 넣고 사용함.

role_arn = "arn:aws:iam::376278017302:role/service-role/AmazonSageMaker-ExecutionRole-20230112T204234"
print(role_arn)

arn:aws:iam::376278017302:role/service-role/AmazonSageMaker-ExecutionRole-20230112T204234


## 2. 데이터 세트 그룹 생성 및 대기

Personalize에서 가장 큰 단위는 **데이터 세트 그룹(Dataset Group)** 이며, 이렇게 하면 데이터, 이벤트 추적기(event tracker), 솔루션(solution) 및 캠페인(campaign)이 분리됩니다. 공통의 데이터 수집을 공유하는 것들을 그룹화합니다. 원하는 경우 아래 그룹명을 자유롭게 변경해 주세요.

### 2.1 데이터 세트 그룹 생성

In [7]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "RetailDemo-dataset-group-" + suffix
)

In [8]:
dataset_group_arn = create_dataset_group_response['datasetGroupArn']
dataset_group_arn

'arn:aws:personalize:us-east-1:376278017302:dataset-group/RetailDemo-dataset-group-39668'

#### 데이터 세트 그룹이 활성화 상태가 될 때까지 대기

아래의 모든 항목에서 Dataset Group을 사용하려면 활성화(active)가 되어야 합니다. 아래 셀을 실행하고 DatasetGroup: ACTIVE로 변경될 때까지 기다려 주세요.

In [9]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

DatasetGroup: ACTIVE


## 3. 스키마 생성

Personalize가 데이터를 이해하는 방법의 핵심 구성 요소는 아래 정의 된 스키마(schema)에서 비롯됩니다. 이 설정은 CSV 파일을 통해 제공된 데이터를 요약하는 방법을 Personalize 서비스에 알려줍니다. 열(column)과 유형(type)은 위에서 만든 파일의 내용과 일치합니다.

### 3.1 인터렉션

In [10]:
interaction_schema_name="RetailDemo-interaction-schema-"+suffix

schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },        
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema( 
    name = interaction_schema_name,
    schema = json.dumps(schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:376278017302:schema/RetailDemo-interaction-schema-39668",
  "ResponseMetadata": {
    "RequestId": "81c1d519-9ec3-48b7-872d-0f527d45549c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:00:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "101",
      "connection": "keep-alive",
      "x-amzn-requestid": "81c1d519-9ec3-48b7-872d-0f527d45549c"
    },
    "RetryAttempts": 0
  }
}


### 3.2 아이템

In [11]:
item_schema_name="RetailDemo-item-schema-"+suffix

item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
      "name": "PRICE",
      "type": "float"
    },
    {
      "name": "CVR",
      "type": "float"
    }, 
    {
      "name": "CATEGORY_L1",
      "type": [
        "string"
      ],
      "categorical": True
    }        
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(      
    name = item_schema_name,
    schema = json.dumps(item_schema)
)

item_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:376278017302:schema/RetailDemo-item-schema-39668",
  "ResponseMetadata": {
    "RequestId": "5ef1e733-b46b-4f3f-82a0-ff9d844b0788",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:03:14 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "94",
      "connection": "keep-alive",
      "x-amzn-requestid": "5ef1e733-b46b-4f3f-82a0-ff9d844b0788"
    },
    "RetryAttempts": 0
  }
}


### 3.3 유저

In [13]:
user_schema_name="RetailDemo-user-schema-"+suffix

user_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "USER_ID",
        "type": "string"
    },
    {
      "name": "USER_NAME",
      "type": "string"
    },        
    {
      "name": "GENDER",
      "type": [
        "string"
      ],
      "categorical": True
    }        
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(      
    name = user_schema_name,
    schema = json.dumps(user_schema)
)

user_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:376278017302:schema/RetailDemo-user-schema-39668",
  "ResponseMetadata": {
    "RequestId": "78a6f9a7-9846-4e28-b0c2-d04dd9d6946a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:04:15 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "94",
      "connection": "keep-alive",
      "x-amzn-requestid": "78a6f9a7-9846-4e28-b0c2-d04dd9d6946a"
    },
    "RetryAttempts": 0
  }
}


## 4. 데이터 세트 생성

그룹 다음으로 생성할 것은 실제 데이터 세트입니다. 아래의 코드 셀을 실행하여 데이터 세트을 생성해 주세요.

### 4.1 Interactions 데이터 세트 생성

In [14]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "RetailDemo-interaction-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interaction_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:376278017302:dataset/RetailDemo-dataset-group-39668/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "c1f3c16b-0f5d-487a-972d-83c8a2010e38",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:05:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "111",
      "connection": "keep-alive",
      "x-amzn-requestid": "c1f3c16b-0f5d-487a-972d-83c8a2010e38"
    },
    "RetryAttempts": 0
  }
}


### 4.2 Items 데이터 세트 생성

In [15]:
dataset_type = "ITEMS"
create_item_dataset_response = personalize.create_dataset(
    name = "RetailDemo-item-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn,
  
)

item_dataset_arn = create_item_dataset_response['datasetArn']
print(json.dumps(create_item_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:376278017302:dataset/RetailDemo-dataset-group-39668/ITEMS",
  "ResponseMetadata": {
    "RequestId": "d0b05d09-f0aa-44a7-ac6a-0ccd003d6976",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:06:05 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "104",
      "connection": "keep-alive",
      "x-amzn-requestid": "d0b05d09-f0aa-44a7-ac6a-0ccd003d6976"
    },
    "RetryAttempts": 0
  }
}


### 4.3 Users 데이터 세트 생성

In [16]:
dataset_type = "USERS"
create_user_dataset_response = personalize.create_dataset(
    name = "RetailDemo-user-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = user_schema_arn,
  
)

user_dataset_arn = create_user_dataset_response['datasetArn']
print(json.dumps(create_user_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:376278017302:dataset/RetailDemo-dataset-group-39668/USERS",
  "ResponseMetadata": {
    "RequestId": "5e9a7aa8-40e3-4bbe-9b40-aad8b42193f2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:06:24 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "104",
      "connection": "keep-alive",
      "x-amzn-requestid": "5e9a7aa8-40e3-4bbe-9b40-aad8b42193f2"
    },
    "RetryAttempts": 0
  }
}


#### 데이터 세트 생성을 위해서 1분 기다림

In [17]:
time.sleep(60)

## 5. 데이터 세트 Import

이전에는 정보를 저장하기 위해 데이터 세트 그룹 및 데이터 세트를 생성했으므로, 
이제는 모델 구축을 위해 S3에서 Amazon Personalize로 데이터를 로드하는 import job을 실행합니다.

#### Interaction 데이터 세트 Import Job 생성

In [24]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "RetailDeom-interaction-dataset-import-" + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, warm_train_interaction_filename)
    },
    roleArn = role_arn
)

interation_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:376278017302:dataset-import-job/RetailDeom-interaction-dataset-import-39668",
  "ResponseMetadata": {
    "RequestId": "182b054d-478d-4f81-9eca-c8f658f86b15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:14:44 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "131",
      "connection": "keep-alive",
      "x-amzn-requestid": "182b054d-478d-4f81-9eca-c8f658f86b15"
    },
    "RetryAttempts": 0
  }
}


#### 아이템 데이터 세트 Import Job 생성

In [25]:
create_item_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "RetailDemo-item-dataset-import-" + suffix,
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

item_dataset_import_job_arn = create_item_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:376278017302:dataset-import-job/RetailDeom-interaction-dataset-import-39668",
  "ResponseMetadata": {
    "RequestId": "182b054d-478d-4f81-9eca-c8f658f86b15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:14:44 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "131",
      "connection": "keep-alive",
      "x-amzn-requestid": "182b054d-478d-4f81-9eca-c8f658f86b15"
    },
    "RetryAttempts": 0
  }
}


#### 유저 데이터 세트 Import Job 생성

In [26]:
create_user_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "RetailDemo-user-dataset-import-" + suffix,
    datasetArn = user_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, users_filename)
    },
    roleArn = role_arn
)

user_dataset_import_job_arn = create_user_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_user_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:376278017302:dataset-import-job/RetailDemo-user-dataset-import-39668",
  "ResponseMetadata": {
    "RequestId": "e9c48bab-901d-404e-8f4b-fa623ad60b74",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 28 Feb 2023 12:15:46 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "124",
      "connection": "keep-alive",
      "x-amzn-requestid": "e9c48bab-901d-404e-8f4b-fa623ad60b74"
    },
    "RetryAttempts": 0
  }
}


### 데이터 세트 Import job이 활성화 상태가 될 때까지 대기

Import job이 완료되기까지 시간이 걸립니다. 아래 코드 셀의 출력 결과가 DatasetImportJob: ACTIVE가 될 때까지 기다려 주세요.

#### 인터렉션

In [27]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interation_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 24.6 ms, sys: 2.81 ms, total: 27.4 ms
Wall time: 2min


#### 아이템

In [28]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = item_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


#### 유저

In [29]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = user_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


## Review

위의 코드를 사용하여 데이타 세트 그룹, 데이타 세트, 데이타 세트 Import까지를 수행 하였습니다. 다음 단계는 이를 기반으로 솔류션(모델)을 생성하는 단계를 진행 합니다.


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [31]:
%store dataset_group_arn
%store interaction_dataset_arn
%store item_dataset_arn
%store user_dataset_arn
%store interaction_schema_arn
%store item_schema_arn
%store user_schema_arn
%store role_arn
%store suffix

Stored 'dataset_group_arn' (str)
Stored 'interaction_dataset_arn' (str)
Stored 'item_dataset_arn' (str)
Stored 'user_dataset_arn' (str)
Stored 'interaction_schema_arn' (str)
Stored 'item_schema_arn' (str)
Stored 'user_schema_arn' (str)
Stored 'role_arn' (str)
Stored 'suffix' (str)
